<a href="https://colab.research.google.com/github/andreaxlei/ACMDIGW-K32/blob/main/ACMDIGW_App_Tan%2C_Andrea_Lei_T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>